In [6]:
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.segmentation import random_walker
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.filters import threshold_multiotsu

# Bird Eye View

In [7]:
def total_top_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(350, int(2*img.shape[0]/3)+55),     # top-left
                       (100, img.shape[0]-100),     # bottom-left
                       (img.shape[1]-100, img.shape[0]-100),    # bottom-right
                       (img.shape[1]-350, int(2*img.shape[0]/3)+55)])    # top-right


    dst = np.float32([(350,0),     # top-left
                       (100, img.shape[0]),     # bottom-left
                       (img.shape[1]-100, img.shape[0]),    # bottom-right
                       (img.shape[1]-350, 0)]) # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)

In [8]:
def lane_top_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(300, 600),     # top-left
                       (300, img.shape[0]-70),     # bottom-left
                       (900, img.shape[0]-70),    # bottom-right
                       (900, 600)])    # top-right

    dst = np.float32([(150, 0),     # top-left
                       (150, img.shape[0]-70),     # bottom-left
                       (1000, img.shape[0]-70),    # bottom-right
                       (1000, 0)])    # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)

In [ ]:
# for i in np.arange(1,2253, 30):
#     print(i)
img = cv2.imread('IMAGE PATH')
mask=img.copy()
mask[:int(2*mask.shape[0]/3)+50,:]=0
total_view=total_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)
total_view = cv2.convertScaleAbs(total_view, alpha=2, beta=25)

fig, ax = plt.subplots(1, 3, figsize=(20, 20))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title('orginal image')

ax[1].imshow(cv2.cvtColor(total_view, cv2.COLOR_BGR2RGB))
ax[1].set_title('total_view')

lane_view=lane_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)
lane_view = cv2.convertScaleAbs(lane_view, alpha=2, beta=25)

ax[2].imshow(cv2.cvtColor(lane_view, cv2.COLOR_BGR2RGB))
ax[2].set_title('lane_view')
plt.show()

# Road-Marks Detection

### Canny method

In [ ]:
def canny(img,d,u):

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = 3
    blur = cv2.GaussianBlur(gray,(kernel, kernel),0)
    canny = cv2.Canny(blur, d, u)
    kernel =np.ones((3, 3),np.uint8)
#     canny = cv2.erode(canny, kernel, iterations=5)
    canny = cv2.dilate(canny, kernel, iterations=1)

    return canny


### Histogram classifier & Random walker

In [10]:
def run(top_view,image,n_class):
    sigma_est = np.mean(estimate_sigma(top_view, multichannel=True))
    denoise_img = denoise_nl_means(top_view, h=1.15 * sigma_est, fast_mode=True, 
                       patch_size=5, patch_distance=3, multichannel=True)


    image = (exposure.equalize_adapthist(denoise_img))

    thresholds = threshold_multiotsu(image, classes=n_class)


    
    regions = np.digitize(image, bins=thresholds)
    
    regions = random_walker(image, regions+1, beta=10, mode='bf')
    output =(255*(regions/(n_class))).astype(np.uint8)  #Convert 64 bit integer values to uint8

    gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    _, output = cv2.threshold(gray, 170, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    
    output = cv2.erode(output, kernel, iterations=2)
    
    return output

### Lab color model

In [ ]:
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1 = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
top_view1=cv2.cvtColor(top_view1, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(top_view1)
L = cv2.convertScaleAbs(L, alpha=1, beta=100)
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax[0].hist(L.ravel(), bins=255,color='blue')
ax[0].set_title('L')
Lthresholds = threshold_multiotsu(L, classes=3)
for thresh in Lthresholds:
    ax[0].axvline(thresh, color='black')

ax[1].hist(a.ravel(), bins=255,color='red')
ax[1].set_title('a')
athresholds = threshold_multiotsu(a, classes=3)
for thresh in athresholds:
    ax[1].axvline(thresh, color='black')

ax[2].hist(b.ravel(), bins=255,color='green')
ax[2].set_title('b')
bthresholds = threshold_multiotsu(b, classes=3)
for thresh in bthresholds:
    ax[2].axvline(thresh, color='black')

ax[3].hist(top_view1.ravel(), bins=255,color='green')
ax[3].set_title('b')
labthresholds = threshold_multiotsu(top_view1, classes=3)
for thresh in labthresholds:
    ax[3].axvline(thresh, color='black')
plt.show()     


fig, ax = plt.subplots(1, 4, figsize=(20, 3))
#     _, output = cv2.threshold(L, 1.1*Lthresholds[0], 255, cv2.THRESH_BINARY_INV)

ax[0].imshow(L,cmap='gray')
ax[0].set_title('L')
ax[0].axis('off')

#     _, output = cv2.threshold(a, 1.1*athresholds[0], 255, cv2.THRESH_BINARY_INV)

ax[1].imshow(a,cmap='gray')
ax[1].set_title('a')
ax[1].axis('off')

#     _, output = cv2.threshold(b, 1.1*bthresholds[-1], 255, cv2.THRESH_BINARY)

ax[2].imshow(b,cmap='gray')
ax[2].set_title('b')
ax[2].axis('off')

ax[3].imshow(top_view1,cmap='gray')
ax[3].set_title('lab')
ax[3].axis('off')

plt.show()

In [ ]:

kernel = np.ones((3, 3), np.uint8)  
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1=cv2.cvtColor(top_view, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(top_view1)


fig, ax = plt.subplots(1, 4, figsize=(20, 3))

ax[0].hist(L.ravel(), bins=255,color='green')
ax[0].set_title('lightness histogram')
Lthresholds = threshold_multiotsu(L, classes=4)
#     for thresh in Lthresholds:
#         ax[0].axvline(thresh, color='black')


b=b/b.max()
cax = ax[1].imshow(b, cmap='hot', interpolation='nearest')
fig.colorbar(cax)

threshold1, white_mark = cv2.threshold(L,Lthresholds[-1], 1, cv2.THRESH_BINARY)
#     threshold1, white_mark = cv2.threshold(L, 100, 255, cv2.THRESH_BINARY + 
#                                             cv2.THRESH_OTSU)
ax[0].axvline(Lthresholds[-1], color='red',lw=3)
#     threshold2, yellow_mark = cv2.threshold(b,bthresholds[-1], 1, cv2.THRESH_BINARY)



# Threshold the image to get only "yellow" colors
yellow_mark = b.copy()
yellow_mark[yellow_mark>0.95]=255
yellow_mark[yellow_mark!=255]=0
threshold2, yellow_mark = cv2.threshold(yellow_mark,bthresholds[-1], 1, cv2.THRESH_BINARY)
#     yellow_mark = cv2.erode(yellow_mark, kernel, iterations=10)
#     yellow_mark = cv2.dilate(yellow_mark, kernel, iterations=10)
output = white_mark + yellow_mark
#     output = cv2.erode(output, kernel, iterations=10)
#     output = cv2.dilate(output, kernel, iterations=1)

ax[2].imshow(white_mark,cmap='gray')
ax[2].set_title('road marks - lightness')
ax[2].axis('off')


ax[3].imshow(yellow_mark,cmap='gray')
ax[3].set_title('road marks - yellowness')
ax[3].axis('off')




plt.show()


### Hsv color model

In [ ]:
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1 = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
#     top_view1 = cv2.bilateralFilter(top_view1,45,10,10)
top_view1=cv2.cvtColor(top_view1, cv2.COLOR_BGR2HSV)
H,S,V = cv2.split(top_view1)



fig, ax = plt.subplots(1, 3, figsize=(20, 3))
ax[0].hist(H.ravel(), bins=255,color='blue')
ax[0].set_title('H')
Hthresholds = threshold_multiotsu(H, classes=3)
for thresh in Hthresholds:
    ax[0].axvline(thresh, color='black')

ax[1].hist(S.ravel(), bins=255,color='red')
ax[1].set_title('S')
Sthresholds = threshold_multiotsu(S, classes=5)
for thresh in Sthresholds:
    ax[1].axvline(thresh, color='black')

ax[2].hist(V.ravel(), bins=255,color='green')
ax[2].set_title('V')
Vthresholds = threshold_multiotsu(V, classes=3)
for thresh in Vthresholds:
    ax[2].axvline(thresh, color='black')

plt.show()     


fig, ax = plt.subplots(1, 3, figsize=(20, 3))
_, output = cv2.threshold(H, 1.1*Hthresholds[0], 255, cv2.THRESH_BINARY_INV)

ax[0].imshow(output,cmap='gray')
ax[0].set_title('top_view')
ax[0].axis('off')

_, output = cv2.threshold(S, 1.1*Sthresholds[0], 255, cv2.THRESH_BINARY_INV)

ax[1].imshow(output,cmap='gray')
ax[1].set_title('top_view')
ax[1].axis('off')

_, output = cv2.threshold(V, 1.1*Vthresholds[-1], 255, cv2.THRESH_BINARY)

ax[2].imshow(output,cmap='gray')
ax[2].set_title('top_view')
ax[2].axis('off')


plt.show()


### YCrCb

In [ ]:
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1 = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
top_view1=cv2.cvtColor(top_view1, cv2.COLOR_BGR2YCrCb)
Y,Cr,Cb = cv2.split(top_view1)



fig, ax = plt.subplots(1, 3, figsize=(20, 3))
ax[0].hist(Y.ravel(), bins=255,color='blue')
ax[0].set_title('Y')
Ythresholds = threshold_multiotsu(Y, classes=3)
for thresh in Ythresholds:
    ax[0].axvline(thresh, color='black')

ax[1].hist(Cr.ravel(), bins=255,color='red')
ax[1].set_title('Cr')
Crthresholds = threshold_multiotsu(Cr, classes=5)
for thresh in Crthresholds:
    ax[1].axvline(thresh, color='black')

ax[2].hist(Cb.ravel(), bins=255,color='green')
ax[2].set_title('Cb')
Cbthresholds = threshold_multiotsu(Cb, classes=3)
for thresh in Cbthresholds:
    ax[2].axvline(thresh, color='black')

plt.show()     


fig, ax = plt.subplots(1, 3, figsize=(20, 3))
ax[0].imshow(Y,cmap='gray')
ax[0].set_title('top_view')
ax[0].axis('off')

_, output = cv2.threshold(Cr, Crthresholds[-1], 255, cv2.THRESH_BINARY)

ax[1].imshow(output,cmap='gray')
ax[1].set_title('top_view')
ax[1].axis('off')


ax[2].imshow(Cb,cmap='gray')
ax[2].set_title('top_view')
ax[2].axis('off')


plt.show()


### Results Comparison

In [ ]:
# for i in np.arange(1,2253, 30):
#     print(i)
img = cv2.imread('IMAGE PATH')
total_view=total_top_view(img, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

contrast_total_view = cv2.convertScaleAbs(total_view, alpha=2, beta=25)

fig, ax = plt.subplots(1, 5, figsize=(20, 20))
plt.subplots_adjust(wspace=0.1)
ax[0].imshow(cv2.cvtColor(contrast_total_view, cv2.COLOR_BGR2RGB))
ax[0].set_title('orginal image')
ax[0].axis('off')

canny_marks=canny(contrast_total_view, 50, 80)

ax[1].imshow(cv2.cvtColor(canny_marks, cv2.COLOR_BGR2RGB))
ax[1].set_title('canny method')
ax[1].axis('off')

image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image = image.astype(np.float64) / 255.0
hist_total_view=total_top_view(image, img_size=(image.shape[1],image.shape[0]), flags=cv2.INTER_LINEAR)
hist_total_view = cv2.convertScaleAbs(hist_total_view, alpha=100, beta=100)
hist_marks=run(hist_total_view,image,3)

ax[2].imshow(cv2.cvtColor(hist_marks, cv2.COLOR_BGR2RGB))
ax[2].set_title('histogram method')
ax[2].axis('off')


lab=cv2.cvtColor(total_view, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(lab)
Lthresholds = threshold_multiotsu(L, classes=4)
th, Lwhite_mark = cv2.threshold(L,Lthresholds[-1], 1, cv2.THRESH_BINARY)

ax[3].imshow(Lwhite_mark,cmap='gray')
ax[3].set_title('Lightness - Lab')
ax[3].axis('off')


contrast_view = cv2.convertScaleAbs(total_view, alpha=2, beta=20)
hsv=cv2.cvtColor(contrast_view, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(contrast_view)


Vthresholds = threshold_multiotsu(v, classes=3)
th, Vwhite_mark = cv2.threshold(v,Vthresholds[-1], 1, cv2.THRESH_BINARY)


ax[4].imshow(Vwhite_mark,cmap='gray')
ax[4].set_title('Value - HSV')
ax[4].axis('off')
plt.show()




# Lane-Marks Detection

### Results Comparison

In [ ]:
def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = cv2.filter2D(src=img, ddepth=-1, kernel=Kx)
    Iy = cv2.filter2D(src=img, ddepth=-1, kernel=Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G.astype(np.uint8), theta)




# for i in np.arange(1,2253, 30):
#     print(i)
img = cv2.imread('IMAGE PATH')
total_view=lane_top_view(img, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

contrast_total_view = cv2.convertScaleAbs(total_view, alpha=2, beta=25)

fig, ax = plt.subplots(1, 5, figsize=(20, 20))
plt.subplots_adjust(wspace=0.1)
ax[0].imshow(cv2.cvtColor(contrast_total_view, cv2.COLOR_BGR2RGB))
ax[0].set_title('orginal image')
ax[0].axis('off')

G, theta=sobel_filters(contrast_total_view)


flattened_arr = G.flatten()
low_tr = np.quantile(flattened_arr, 0.95)
high_tr = np.quantile(flattened_arr, 0.99)
canny_marks=canny(contrast_total_view,low_tr,high_tr)

ax[1].imshow(cv2.cvtColor(canny_marks, cv2.COLOR_BGR2RGB))
ax[1].set_title('canny method')
ax[1].axis('off')

image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image = image.astype(np.float64) / 255.0
hist_total_view=lane_top_view(image, img_size=(image.shape[1],image.shape[0]), flags=cv2.INTER_LINEAR)
hist_total_view = cv2.convertScaleAbs(hist_total_view, alpha=100, beta=100)
hist_marks=run(hist_total_view,image,3)

ax[2].imshow(cv2.cvtColor(hist_marks, cv2.COLOR_BGR2RGB))
ax[2].set_title('histogram method')
ax[2].axis('off')


lab=cv2.cvtColor(total_view, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(lab)
Lthresholds = threshold_multiotsu(L, classes=4)
th, Lwhite_mark = cv2.threshold(L,Lthresholds[-1], 1, cv2.THRESH_BINARY)

ax[3].imshow(Lwhite_mark,cmap='gray')
ax[3].set_title('Lightness - Lab')
ax[3].axis('off')


contrast_view = cv2.convertScaleAbs(total_view, alpha=2, beta=20)
hsv=cv2.cvtColor(contrast_view, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(contrast_view)


Vthresholds = threshold_multiotsu(v, classes=3)
th, Vwhite_mark = cv2.threshold(v,Vthresholds[-1], 1, cv2.THRESH_BINARY)


ax[4].imshow(Vwhite_mark,cmap='gray')
ax[4].set_title('Value - HSV')
ax[4].axis('off')
plt.show()




# Final Results

In [ ]:
def lane_top_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(300, 600),     # top-left
                       (300, img.shape[0]-70),     # bottom-left
                       (900, img.shape[0]-70),    # bottom-right
                       (900, 600)])    # top-right

    dst = np.float32([(150, 0),     # top-left
                       (150, img.shape[0]-70),     # bottom-left
                       (1000, img.shape[0]-70),    # bottom-right
                       (1000, 0)])    # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)




def tr_back_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(300, 600),     # top-left
                       (300, img.shape[0]-70),     # bottom-left
                       (900, img.shape[0]-70),    # bottom-right
                       (900, 600)])    # top-right

    dst = np.float32([(150, 0),     # top-left
                       (150, img.shape[0]-70),     # bottom-left
                       (1000, img.shape[0]-70),    # bottom-right
                       (1000, 0)])    # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)

    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return cv2.warpPerspective(img, M_inv, img_size, flags=flags)

    

def canny(img,d,u):

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = 3
    blur = cv2.GaussianBlur(gray,(kernel, kernel),0)
    canny = cv2.Canny(blur, d, u)
    kernel =np.ones((3, 3),np.uint8)
    
    canny = cv2.dilate(canny, kernel, iterations=4)
    canny = cv2.erode(canny, kernel, iterations=2)
    return canny


def average_slope_intercept(image, lines):
    left_fit    = []
    right_fit   = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1,x2), (y1,y2), 1)
            slope = fit[0]
#             print(slope)
            intercept = fit[1]
            if slope < -5: 
                left_fit.append((slope, intercept))
            if slope > 5:
                right_fit.append((slope, intercept))
    left_fit_average  = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line  = make_points(image, left_fit_average)
    right_line = make_points(image, right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines

def make_points(image, line):
    slope, intercept = line
    y1 = int(image.shape[0])
    y2 = int(y1*3.0/5)      
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]
 




def region_of_interest_lane(img):
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros_like(img)
    poly = np.array([[
    (200,700),
    (200, 0),
    
    (1000, 0),
    (1000, 700)]], np.int32)
    kernel =np.ones((3, 3),np.uint8)
    cv2.fillPoly(mask, poly, (255, 255, 255))
#     mask = cv2.erode(mask, kernel, iterations=50)
#     mask = cv2.dilate(mask, kernel, iterations=1)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def display_advlines(img,lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),10)
    return line_image



In [ ]:
def extract_features(img , nwindows = 9):
    # Height of of windows - based on nwindows and image shape
    window_height = np.int(img.shape[0]//nwindows)

    # Identify the x and y positions of all nonzero pixel in the image
    nonzero = img.nonzero()
    nonzerox = np.array(nonzero[1])
    nonzeroy = np.array(nonzero[0])
    
    return window_height , nwindows ,nonzero , nonzerox ,nonzeroy

def pixels_in_window(center, margin, height):  

    topleft = (center[0]-margin, center[1]-height//2)
    bottomright = (center[0]+margin, center[1]+height//2)

    condx = (topleft[0] <= nonzerox) & (nonzerox <= bottomright[0])
    condy = (topleft[1] <= nonzeroy) & (nonzeroy <= bottomright[1])
    
    return nonzerox[condx&condy], nonzeroy[condx&condy]


margin = 100
minpix = 50

def find_lane_pixels(img):

    assert(len(img.shape) == 2)
    out_img = np.dstack((img, img, img))

    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    midpoint = histogram.shape[0]//2
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Current position to be update later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    y_current = img.shape[0] + window_height//2

    # Create empty lists to reveice left and right lane pixel
    leftx, lefty, rightx, righty = [], [], [], []

    # Step through the windows one by one
    for _ in range(nwindows):
        y_current -= window_height
        center_left = (leftx_current, y_current)
        center_right = (rightx_current, y_current)

        good_left_x, good_left_y = pixels_in_window(center_left, margin, window_height)
        good_right_x, good_right_y = pixels_in_window(center_right, margin, window_height)

        # Append these indices to the lists
        leftx.extend(good_left_x)
        lefty.extend(good_left_y)
        rightx.extend(good_right_x)
        righty.extend(good_right_y)

        if len(good_left_x) > minpix:
            leftx_current = np.int32(np.mean(good_left_x))
        if len(good_right_x) > minpix:
            rightx_current = np.int32(np.mean(good_right_x))

    return leftx, lefty, rightx, righty, out_img


left_fit = None
right_fit = None

def fit_poly(img):

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img)

    
    if len(lefty) > 1500:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(righty) > 1500:
        right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    maxy = img.shape[0] - 1
    miny = img.shape[0] // 3
    
    if len(lefty):
        maxy = max(maxy, np.max(lefty))
        miny = min(miny, np.min(lefty))

    if len(righty):
        maxy = max(maxy, np.max(righty))
        miny = min(miny, np.min(righty))

    ploty = np.linspace(miny, maxy, img.shape[0]-450)

    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
#         left_fitx = left_fit[0]*ploty + left_fit[1]
#         right_fitx = right_fit[0]*ploty + right_fit[1]

        # Visualization
        for i, y in enumerate(ploty):
            l = int(left_fitx[i])
            r = int(right_fitx[i])
            y = int(y)
            cv2.line(out_img, (l, y), (r, y), (0, 255, 0))

        return out_img ,True,left_fit,right_fit
    except:
        return out_img ,False ,None,None



def lane_top_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(300, 600),     # top-left
                       (300, img.shape[0]-70),     # bottom-left
                       (900, img.shape[0]-70),    # bottom-right
                       (900, 600)])    # top-right

    dst = np.float32([(150, 0),     # top-left
                       (150, img.shape[0]-70),     # bottom-left
                       (1000, img.shape[0]-70),    # bottom-right
                       (1000, 0)])    # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)




def lane_back_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(300, 600),     # top-left
                       (300, img.shape[0]-70),     # bottom-left
                       (900, img.shape[0]-70),    # bottom-right
                       (900, 600)])    # top-right

    dst = np.float32([(150, 0),     # top-left
                       (150, img.shape[0]-70),     # bottom-left
                       (1000, img.shape[0]-70),    # bottom-right
                       (1000, 0)])    # top-right

    
    
    M = cv2.getPerspectiveTransform(src, dst)

    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return cv2.warpPerspective(img, M_inv, img_size, flags=flags)


def total_top_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(350, int(2*img.shape[0]/3)+55),     # top-left
                       (100, img.shape[0]-100),     # bottom-left
                       (img.shape[1]-100, img.shape[0]-100),    # bottom-right
                       (img.shape[1]-350, int(2*img.shape[0]/3)+55)])    # top-right


    dst = np.float32([(350,0),     # top-left
                       (100, img.shape[0]),     # bottom-left
                       (img.shape[1]-100, img.shape[0]),    # bottom-right
                       (img.shape[1]-350, 0)])

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)
    

def total_back_view(img, img_size, flags=cv2.INTER_LINEAR):

    src = np.float32([(350, int(2*img.shape[0]/3)+55),     # top-left
                       (100, img.shape[0]-100),     # bottom-left
                       (img.shape[1]-100, img.shape[0]-100),    # bottom-right
                       (img.shape[1]-350, int(2*img.shape[0]/3)+55)])    # top-right


    dst = np.float32([(350,0),     # top-left
                       (100, img.shape[0]),     # bottom-left
                       (img.shape[1]-100, img.shape[0]),    # bottom-right
                       (img.shape[1]-350, 0)])

    
    
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return cv2.warpPerspective(img, M_inv, img_size, flags=flags)

def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = cv2.filter2D(src=img, ddepth=-1, kernel=Kx)
    Iy = cv2.filter2D(src=img, ddepth=-1, kernel=Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G.astype(np.uint8), theta)    



    
li=[2,3,4,5,6,7,8,48, 54, 58, 155 ,190, 192 ,195 ,205, 221 ,291 ,296, 297]

is_roi=False
kernel =np.ones((3, 3),np.uint8)

# keep_straight_img = mpimg.imread('straight.png')
start=True
# for i in tqdm(np.arange(1,2252)):
# #     print(i)
img = cv2.imread('IMAGE PATH')

top_view=lane_top_view(img, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

top_view = cv2.convertScaleAbs(top_view, alpha=2, beta=25)


G, theta=sobel_filters(top_view)


flattened_arr = G.flatten()
low_tr = np.quantile(flattened_arr, 0.95)
high_tr = np.quantile(flattened_arr, 0.99)
lane_filtered=canny(top_view,low_tr,high_tr)
#     lab=cv2.cvtColor(top_view, cv2.COLOR_BGR2Lab)
#     L,a,b = cv2.split(lab)
#     Lthresholds = threshold_multiotsu(L, classes=4)
#     th, lane_filtered = cv2.threshold(L,Lthresholds[-1], 255, cv2.THRESH_BINARY)

if is_roi:

    ployy = cv2.dilate(ployy, kernel, iterations=150)
    _, roi = cv2.threshold(ployy, 170, 1, cv2.THRESH_BINARY)
    lane_filtered =lane_filtered * roi
window_height , nwindows ,nonzero , nonzerox ,nonzeroy = extract_features(lane_filtered , nwindows = 9)
ployy,is_roi,left_fit,right_fit=fit_poly(lane_filtered)

if not is_roi:
    lane_filtered=canny(top_view,low_tr,high_tr)
    window_height , nwindows ,nonzero , nonzerox ,nonzeroy = extract_features(lane_filtered , nwindows = 9)
    ployy,is_roi,left_fit,right_fit=fit_poly(lane_filtered)



ployy=ployy[:,:,1]-ployy[:,:,0]



top_view=total_top_view(img, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

lab=cv2.cvtColor(top_view, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(lab)
Lthresholds = threshold_multiotsu(L, classes=4)
th, Lwhite_mark = cv2.threshold(L,Lthresholds[-1], 255, cv2.THRESH_BINARY)

contrast_view = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
hsv=cv2.cvtColor(contrast_view, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(contrast_view)


Vthresholds = threshold_multiotsu(v, classes=4)
th, Vwhite_mark = cv2.threshold(v,Vthresholds[-1], 1, cv2.THRESH_BINARY)

mask= Lwhite_mark + Vwhite_mark
_ , mask=cv2.threshold(mask,0.5, 1, cv2.THRESH_BINARY)
kernel =np.ones((3, 3),np.uint8)
mask = cv2.dilate(mask, kernel, iterations=1)
raw=np.zeros_like(top_view)
raw[mask!=0]=[0,0,255]
back_view=total_back_view(raw, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)
out_img = cv2.addWeighted(back_view, 1, img, 1, 0)

#     print(is_roi,start,ployy.mean())
if ployy.mean()<15:
    out_img = cv2.addWeighted(np.stack((np.zeros_like(lastframe_back),lastframe_back,np.zeros_like(lastframe_back)), axis=-1), 1, out_img, 1, 0)
else:
    back_view=lane_back_view(ployy, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)
    out_img = cv2.addWeighted(np.stack((np.zeros_like(back_view),back_view,np.zeros_like(back_view)), axis=-1), 1, out_img, 1, 0)
    lastframe_back=back_view.copy()
    start=False





fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(lane_filtered, cv2.COLOR_BGR2RGB))
plt.show()    


# Additional Tests

### Region Growing > Road Segmentation

In [ ]:
import cv2
import numpy as np

def get_neighbouring_pixels(x, y, shape):
    neighbours = []
    max_x = shape[1] - 1
    max_y = shape[0] - 1
    directions = [(-1, -1), (0, -1), (1, -1), (-1, 0), (1, 0), (-1, 1), (0, 1), (1, 1)]

    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx <= max_x and 0 <= ny <= max_y:
            neighbours.append((nx, ny))

    return neighbours


def euclidean_distance(a, b):
    return np.sqrt(np.sum((a-b)**2))

def region_growing(img, seed, threshold):
    seed_points = [seed]
    out_img = np.zeros_like(img)
    processed = []

    while seed_points:
        pix = seed_points.pop(0)
        out_img[pix] = img[pix]

        for coord in get_neighbouring_pixels(*pix, img.shape):
            try:
                if euclidean_distance(img[seed], img[coord]) < threshold and coord not in processed:
                    out_img[coord] = img[coord]
                    seed_points.append(coord)
                    processed.append(coord)
            except:
                pass

    return out_img

image = cv2.imread('IMAGE PATH')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
image=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   
    
image = cv2.resize(image, (0,0), fx=0.1, fy=0.1)
seed = (50,85)
color_threshold = 15.0  # adjust this value based on your requirements
out = region_growing(image, seed, color_threshold)

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# cv2.imshow('image',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
ax[0].imshow(image,cmap='gray')
ax[1].imshow(out,cmap='gray')
plt.show()

### K-means Clustering > Road Marks Detection

In [ ]:
i=0
is_roi=False

kernel =np.ones((3, 3),np.uint8)


# for i in np.arange(1,2000,50):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
mask=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1 = cv2.convertScaleAbs(mask, alpha=2, beta=20)
top_view1 = cv2.bilateralFilter(top_view1,45,10,10)
top_view=cv2.cvtColor(top_view1, cv2.COLOR_BGR2HSV)
H,S,V = cv2.split(top_view1)


fig, ax = plt.subplots(1, 2, figsize=(20, 5))
# # GaussianBlur(img,(5,5),0)

ax[0].imshow(cv2.cvtColor(top_view1, cv2.COLOR_BGR2RGB))
ax[0].set_title('top_view')
ax[0].axis('off')
pixel_vals = V.reshape((-1,1))
pixel_vals = np.float32(pixel_vals)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
retval, labels, centers = cv2.kmeans(pixel_vals, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
segmented_data = centers[labels.flatten()]
segmented_image = segmented_data.reshape((V.shape))
#     ret, lane_filtered = cv2.threshold(segmented_image, 100, 255, cv2.THRESH_BINARY)
ret, lane_filtered = cv2.threshold(segmented_image, 100, 255, cv2.THRESH_BINARY + 
                                        cv2.THRESH_OTSU)
#     print(ret)
#     ax[1].imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
#     ax[1].set_title('top_view')
#     ax[1].axis('off')


ax[1].imshow(cv2.cvtColor(lane_filtered, cv2.COLOR_BGR2RGB))
ax[1].set_title('kmeans 2 cluster')
ax[1].axis('off')
plt.show()




In [ ]:
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
top_view1=cv2.cvtColor(top_view, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(top_view1)


fig, ax = plt.subplots(1, 5, figsize=(20, 10))

ax[0].imshow(cv2.cvtColor(top_view, cv2.COLOR_BGR2RGB))
ax[0].set_title('image')
ax[0].axis('off')

ax[1].imshow(b,cmap='gray')
ax[1].set_title('b')
ax[1].axis('off')


pixel_vals = b.reshape((-1,1))
pixel_vals = np.float32(pixel_vals)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
retval, labels, centers = cv2.kmeans(pixel_vals, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
segmented_data = centers[labels.flatten()]
segmented_image = segmented_data.reshape((b.shape))

ret, output = cv2.threshold(segmented_image, 100, 255, cv2.THRESH_BINARY + 
                                        cv2.THRESH_OTSU)
ax[2].imshow(output,cmap='gray')
ax[2].set_title('kmean - 2cluster')
ax[2].axis('off')



ret, output = cv2.threshold(b, 100, 255, cv2.THRESH_BINARY + 
                                        cv2.THRESH_OTSU)

ax[3].imshow(output,cmap='gray')
ax[3].set_title('b - THRESH_OTSU')
ax[3].axis('off')

fil,gi =run(top_view,image,2)
ax[4].imshow(fil,cmap='gray')
ax[4].set_title('contrast - randomwalker')
ax[4].axis('off')

plt.show()




### Lab color model > Road Segmentation

In [ ]:
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1 = cv2.convertScaleAbs(top_view, alpha=2, beta=20)
top_view1=cv2.cvtColor(top_view1, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(top_view1)

fig, ax = plt.subplots(1, 4, figsize=(20, 3))

ax[0].hist(b.ravel(), bins=255,color='blue')
ax[0].set_title('b')
bthresholds = threshold_multiotsu(b, classes=3)
for thresh in bthresholds:
    ax[0].axvline(thresh, color='black')

ax[1].imshow(b,cmap='gray')
ax[1].set_title('top_view')
ax[1].axis('off')

#     _, output = cv2.threshold(b, bthresholds[0], 255, cv2.THRESH_BINARY)


ret, output = cv2.threshold(b, 100, 255, cv2.THRESH_BINARY + 
                                        cv2.THRESH_OTSU)
ax[2].imshow(output,cmap='gray')
ax[2].set_title('top_view')
ax[2].axis('off')

ax[3].imshow(cv2.cvtColor(top_view, cv2.COLOR_BGR2RGB))
ax[3].set_title('top_view')
ax[3].axis('off')

ax[0].axvline(ret, color='red')
plt.show()
        

In [ ]:
kernel = np.ones((3, 3), np.uint8)  
# for i in np.arange(1,89,5):
#     print(i)
image = cv2.imread('IMAGE PATH')

mask=image.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(mask.shape[1],mask.shape[0]), flags=cv2.INTER_LINEAR)   

top_view1=cv2.cvtColor(top_view, cv2.COLOR_BGR2Lab)
top_view1 = cv2.bilateralFilter(top_view1,45,10,10)
L,a,b = cv2.split(top_view1)



Lthresholds = threshold_multiotsu(L, classes=4)

threshold1, marks = cv2.threshold(L,Lthresholds[-1], 1, cv2.THRESH_BINARY)

marks = cv2.erode(marks, kernel, iterations=1)
marks = cv2.dilate(marks, kernel, iterations=2)


mask = cv2.dilate(marks, kernel, iterations=12)
road_seg=top_view.copy()
road_seg[mask!=0]=0
road_seg=cv2.cvtColor(road_seg, cv2.COLOR_BGR2Lab)
L,a,b = cv2.split(road_seg)
ret, road = cv2.threshold(b, 100, 255, cv2.THRESH_BINARY + 
                                        cv2.THRESH_OTSU)

road = cv2.erode(road, kernel, iterations=5)
road = cv2.dilate(road, kernel, iterations=10)

fig, ax = plt.subplots(1, 3, figsize=(20, 3))


ax[0].imshow(marks,cmap='gray')
ax[0].set_title('all marks')
ax[1].imshow(road,cmap='gray')
ax[1].set_title('road segmentation')

finall_img=np.zeros_like(top_view)
finall_img[road==0]=[100,255,255]

cax=cv2.addWeighted(finall_img, 0.5, top_view, 1, 0)

cax[marks!=0]=[0,0,255]

ax[2].imshow(cv2.cvtColor(cax, cv2.COLOR_BGR2RGB))
ax[2].set_title('finall')


plt.show()
    

### Hough lane detaction

In [ ]:

def region_of_interest_road(img):
    height = img.shape[0]
    width = img.shape[1]
    mask = 255*np.ones_like(img)
    poly = np.array([[
    (500,1080),
    (600, 0),
    
    (1200, 0),
    (1300, 1080)]], np.int32)
    kernel =np.ones((3, 3),np.uint8)
    cv2.fillPoly(mask, poly, (0, 0, 0))
#     mask = cv2.erode(mask, kernel, iterations=50)
#     mask = cv2.dilate(mask, kernel, iterations=1)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def region_of_interest_lane(img):
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros_like(img)
    poly = np.array([[
    (500,1000),
    (650, 500),
    
    (1200, 500),
    (1400, 1000)]], np.int32)
    kernel =np.ones((3, 3),np.uint8)
    cv2.fillPoly(mask, poly, (255, 255, 255))
#     mask = cv2.erode(mask, kernel, iterations=50)
#     mask = cv2.dilate(mask, kernel, iterations=1)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



alpha=2
beta=10




# for i in np.arange(1,133,5):
img = cv2.imread('IMAGE PATH')
mask=img.copy()
mask[:int(2*mask.shape[0]/3),:]=0
mask[1000:,:]=0
top_view=tr_top_view(mask, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)   
top_view = cv2.convertScaleAbs(top_view, alpha=alpha, beta=beta)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
top_view = cv2.filter2D(src=top_view, ddepth=-1, kernel=kernel)


fig, ax = plt.subplots(1, 4, figsize=(20, 20))
ax[0].imshow(cv2.cvtColor(top_view, cv2.COLOR_BGR2RGB))
ax[0].set_title('top_view')   

edges=canny(top_view, 30, 60)
kernel =np.ones((3, 3),np.uint8)
edges = cv2.erode(edges, kernel, iterations=10)
edges = cv2.dilate(edges, kernel, iterations=10)

ax[1].imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))
ax[1].set_title('hough') 

try:
    edges = region_of_interest_road(edges)
    advlines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, np.array([]), minLineLength=100, maxLineGap=2)
    lanes = average_slope_intercept(top_view.copy(),advlines)
    lanes=display_advlines(mask.copy(),lanes)

    ax[2].imshow(cv2.cvtColor(cv2.addWeighted(lanes, 1, top_view, 0.6, 0), cv2.COLOR_BGR2RGB))
    ax[2].set_title('finall hough')
except:
    ax[2].imshow(cv2.cvtColor(top_view, cv2.COLOR_BGR2RGB))
    ax[2].set_title('finall hough')
try:
    edges=canny(top_view, 25, 50)
    edges = region_of_interest_lane(edges)
    advlines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, np.array([]), minLineLength=100, maxLineGap=2)
    lanes = average_slope_intercept(top_view.copy(),advlines)
    lanes=display_advlines(mask.copy(),lanes)

    ax[3].imshow(cv2.cvtColor(cv2.addWeighted(lanes, 1, top_view, 0.6, 0), cv2.COLOR_BGR2RGB))
    ax[3].set_title('finall hough')
except:
    ax[3].imshow(cv2.cvtColor(top_view, cv2.COLOR_BGR2RGB))
    ax[3].set_title('finall hough')

plt.show()




### Traffic Sign Detection

In [ ]:
kernel =np.ones((3, 3),np.uint8)
def region_of_interest(img,is_left):
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros_like(img)
    
    if is_left:
        poly = np.array([[
        (0,int(0.75*height)),
        (0, int(0.3*height)),

        (int(width/3), int(0.3*height)),
        (int(width/3), int(0.75*height))]], np.int32)
        
    else:
        poly = np.array([[
        (int(2*width/3),int(0.75*height)),
        (int(2*width/3), int(0.3*height)),

        (width, int(0.3*height)),
        (width, int(0.75*height))]], np.int32)
        
    cv2.fillPoly(mask, poly, (255, 255, 255))
#     mask = cv2.erode(mask, kernel, iterations=50)
#     mask = cv2.dilate(mask, kernel, iterations=1)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def right_view(img, img_size, flags=cv2.INTER_LINEAR):
    width = img.shape[1]
    height = img.shape[0]
    src = np.float32([(int(2*width/3), int(0.3*height)),     # top-left
                       (int(2*width/3),int(0.75*height)),     # bottom-left
                       (width, int(0.75*height)),    # bottom-right
                       (width, int(0.3*height))])    # top-right


    dst = np.float32([(0,0),     # top-left
                       (0, img.shape[0]),     # bottom-left
                       (img.shape[1], img.shape[0]),    # bottom-right
                       (img.shape[1], 0)])

    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    return cv2.warpPerspective(img, M, img_size, flags=flags)


def canny(img,d,u):

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = 3
    blur = cv2.GaussianBlur(gray,(kernel, kernel),0)
    canny = cv2.Canny(blur, d, u)
    kernel =np.ones((3, 3),np.uint8)
#     canny = cv2.erode(canny, kernel, iterations=5)
    canny = cv2.dilate(canny, kernel, iterations=1)

    return canny



def roi(img):
    height = img.shape[0]
    width = img.shape[1]
    mask = np.zeros_like(img)
    

    poly = np.array([[
    (0,int(0.75*height)),
    (0, int(0.3*height)),

    (width, int(0.3*height)),
    (width, int(0.75*height))]], np.int32)
    cv2.fillPoly(mask, poly, (255, 255, 255))
#     mask = cv2.erode(mask, kernel, iterations=50)
#     mask = cv2.dilate(mask, kernel, iterations=1)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:

# for i in [4,102,143,144,176,177,189,193,251,410,457,1895,1908,1926,1978,2000]:
#     print(i)
img = cv2.imread('IMAGE PATH')
total_view = roi(img)
#     total_view=right_view(img, img_size=(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

contrast_total_view = cv2.convertScaleAbs(total_view, alpha=2, beta=25)
fig, ax = plt.subplots(1, 4, figsize=(20, 20))
plt.subplots_adjust(wspace=0.1)
ax[0].imshow(cv2.cvtColor(contrast_total_view, cv2.COLOR_BGR2RGB))
ax[0].set_title('orginal image')
ax[0].axis('off')



canny_marks=canny(contrast_total_view, 50, 80)

ax[1].imshow(cv2.cvtColor(canny_marks, cv2.COLOR_BGR2RGB))
ax[1].set_title('canny method')
ax[1].axis('off')

gray = cv2.cvtColor(contrast_total_view, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
#     binary = cv2.erode(binary, kernel, iterations=50)
ax[2].imshow(binary, cmap='gray')
ax[2].set_title('thresh image')


contours,_ = cv2.findContours(canny_marks, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    # Get convex hull
    hull = cv2.convexHull(cnt)

    # Get approximate polygon
    epsilon = 0.02*cv2.arcLength(hull,True)
    approx = cv2.approxPolyDP(hull,epsilon,True)

#         if len(approx) == 3:
        # Draw triangle
#             cv2.drawContours(contrast_total_view,[approx],0,(0,255,0),10)
    if len(approx) == 4:
        # Draw rectangle
        cv2.drawContours(contrast_total_view,[approx],0,(255,0,0),10)

ax[3].imshow(cv2.cvtColor(contrast_total_view, cv2.COLOR_BGR2RGB))
ax[3].set_title('hough image')



plt.show()
